In [1]:
import os
import pandas as pd

In [2]:
folder_path = 'Forecasts'  # Replace with your folder path
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]


In [3]:
districts = ['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN']

In [4]:
dfs = []  # list to collect all df_req

for f in csv_files:
    df = pd.read_csv(f"{folder_path}/{f}")
    model = f.split('_')[0]
    
    try:
        df_req = df[['district', 'date', 'actual', 'forecast']]
    except KeyError:
        if model == 'bayesian':
            df_req = df[['district', 'date', 'actual', 'forecast_mean']]
            df_req = df_req.rename(columns={'forecast_mean': 'forecast'})  # optional: standardize column name
    df_req['model'] = model 
    dfs.append(df_req)  # collect the processed df_req

# After the loop, concatenate all collected dfs
final_df = pd.concat(dfs, ignore_index=True)

            
        
    # print(df_req.shape)

C:\Users\nauti\AppData\Local\Temp\ipykernel_46796\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model
C:\Users\nauti\AppData\Local\Temp\ipykernel_46796\2311738704.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['model'] = model


In [5]:
final_df.columns

Index(['district', 'date', 'actual', 'forecast', 'model'], dtype='object')

In [6]:
final_df = final_df[['model','district', 'date', 'actual', 'forecast']]

In [7]:
final_df['model'] = final_df['model'].str.upper()  # Convert district names to uppercase

In [8]:
final_df.columns

Index(['model', 'district', 'date', 'actual', 'forecast'], dtype='object')

In [9]:
import numpy as np
import pandas as pd

# Assuming 'final_df' already loaded, with columns: ['district', 'date', 'actual', 'forecast', 'model']

# Empty list to collect results
results = []

districts = final_df['district'].unique()
models = final_df['model'].unique()

for district in districts:
    for model in models:
        temp_df = final_df[(final_df['district'] == district) & (final_df['model'] == model)]
        if not temp_df.empty:
            rmse = np.sqrt(((temp_df['forecast'] - temp_df['actual'])**2).mean())
            results.append({
                'district': district,
                'model': model,
                'RMSE': rmse
            })

# Create DataFrame
rmse_df = pd.DataFrame(results)

# Sort if you want
rmse_df = rmse_df.sort_values(by=['district', 'RMSE'])

# See top results
print(rmse_df.head())


      district   model       RMSE
17  AHMEDNAGAR  SARIMA  47.241465
18  AHMEDNAGAR   TBATS  55.725705
6   AHMEDNAGAR     EXP  58.619809
13  AHMEDNAGAR    LSTM  64.807089
5   AHMEDNAGAR      ET  75.739043


In [10]:
rmse_df[rmse_df.district == "THANE"].sort_values(by='RMSE', ascending=True)

,district,model,RMSE
681,THANE,SARIMA,147.533486
671,THANE,EXP,171.080022
682,THANE,TBATS,171.700118
683,THANE,XGBOOST,188.334314
670,THANE,ET,222.276622
680,THANE,RIDGE,237.731461
675,THANE,LASSO,237.760748
669,THANE,ELASTICNET,242.095173
677,THANE,LINEAR,242.127446
668,THANE,DT,243.375265


In [11]:
final_df['model'].unique()

array(['ANN', 'ARIMA', 'BAYESIAN', 'DT', 'ELASTICNET', 'ET', 'EXP', 'GB',
       'HUBER', 'KNN', 'LASSO', 'LGBM', 'LINEAR', 'LSTM', 'PROPHET', 'RF',
       'RIDGE', 'SARIMA', 'TBATS', 'XGBOOST'], dtype=object)

In [12]:
pivot_df = rmse_df.pivot(index='district', columns='model', values='RMSE')
print(pivot_df)

model                   ANN       ARIMA    BAYESIAN          DT  ELASTICNET  \
district                                                                      
AHMEDNAGAR       108.881629   91.875613   93.433481   91.692174   86.459827   
AKOLA            119.307028   73.872301   67.458159  139.676709   62.995440   
AMRAVATI                NaN  148.886267  416.923628  315.672295  435.600109   
AURANGABAD              NaN  175.506416  326.658386  662.820712  330.856543   
BEED             116.844019  145.280463  111.555790   59.478769   71.464789   
BHANDARA          79.661767   68.396394   53.240374   63.581333   55.437781   
BULDHANA          48.689003   50.066425   44.563284   55.143964   43.753492   
CHANDRAPUR              NaN   86.718562  214.025344  310.585575  220.911148   
DHULE            151.888580  148.578780  119.714471   98.357404  108.183152   
GADCHIROLI              NaN   47.939407  104.054289   99.369000   86.671298   
GONDIA                  NaN   62.270707  132.880879 

In [13]:
pivot_df.to_csv('I48_rmse_pivot.csv', index=True)

In [14]:
# Get the best model for each district (lowest RMSE)
best_models = rmse_df.loc[rmse_df.groupby('district')['RMSE'].idxmin()]

# Display the result
best_models[['district', 'model', 'RMSE']].to_csv('I48_best_models.csv', index=False)